# 🌳 Arbor Notebook Demo - Quick Start

This notebook demonstrates how to use Arbor in Jupyter notebooks for fine-tuning language models with a Ray-like interface.

**Works in:**
- 📓 Local Jupyter Lab/Notebook
- ☁️ Google Colab 
- 🧪 VS Code Notebooks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ziems/arbor/blob/main/examples/notebook_quickstart.ipynb)

## 📦 Installation

First, let's install Arbor using `uv` for faster package management. This may take a few minutes as it installs the ML dependencies, but uv is significantly faster than pip.

**Note:** If you don't have `uv` installed locally, you can use `pip` instead by replacing `uv pip install --system` with `pip install`.

In [ ]:
# Install uv for faster package management
!curl -LsSf https://astral.sh/uv/install.sh | sh
import os
os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"

# Install Arbor from the colab-support branch (includes new Colab integration)
!uv pip install --system git+https://github.com/Ziems/arbor.git@main git+https://github.com/stanfordnlp/dspy.git@main openai 

# Optional: Install flash attention for faster inference (takes ~15 min)
# !uv pip install --system flash-attn --no-build-isolation

## 🚀 Start Arbor Server

Start Arbor in the background. This will:
- Auto-detect available GPUs
- Create a config file automatically
- Start the server in a background thread
- Provide an OpenAI-compatible API

In [ ]:
import arbor

# Start Arbor server in the background
server_info = arbor.init()
print(f"Server running at: {server_info['base_url']}")

## 🔌 Connect with OpenAI Client

Use the standard OpenAI Python client to interact with Arbor.

In [ ]:
# Option 1: Use the convenience method
client = arbor.get_client()

# Option 2: Manual setup
# from openai import OpenAI
# client = OpenAI(
#     base_url=server_info['base_url'],
#     api_key="not-needed"  # Local server doesn't need API key
# )

## 💬 Test Inference

Let's test inference with a small model to make sure everything works.

In [ ]:
# Test chat completion
response = client.chat.completions.create(
    model="HuggingFaceTB/SmolLM2-135M-Instruct",  # Small model for testing
    messages=[
        {"role": "user", "content": "Hello! Tell me a joke about machine learning."}
    ],
    temperature=0.7,
    max_tokens=100
)

print("🤖 Response:", response.choices[0].message.content)

The inference job for that model will persist as long as we want, but we won't be doing inference with that model anymore so let's go ahead and shut that job down

In [ ]:
arbor.shutdown_job("HuggingFaceTB/SmolLM2-135M-Instruct")

## 📁 Upload Training Data

Let's create some sample training data and upload it.

In [ ]:
import json
import tempfile
import os

# Create sample SFT training data
sample_data = [
    {
        "messages": [
            {"role": "user", "content": "What is machine learning?"},
            {"role": "assistant", "content": "Machine learning is a subset of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Explain neural networks simply."},
            {"role": "assistant", "content": "Neural networks are computing systems inspired by biological neural networks. They consist of interconnected nodes (neurons) that process information and learn patterns from data."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "What is deep learning?"},
            {"role": "assistant", "content": "Deep learning is a type of machine learning that uses neural networks with multiple layers to model and understand complex patterns in data."}
        ]
    }
]

# Save to temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.jsonl', delete=False) as f:
    for item in sample_data:
        f.write(json.dumps(item) + '\n')
    temp_file = f.name

print(f"Created training data: {temp_file}")

In [ ]:
# Upload the training file
with open(temp_file, 'rb') as f:
    file_response = client.files.create(
        file=f,
        purpose='fine-tune'
    )

print(f"📁 File uploaded: {file_response.id}")
training_file_id = file_response.id

## 🎯 Start Fine-tuning Job

Now let's start a supervised fine-tuning (SFT) job.

In [ ]:
# Create fine-tuning job
job = client.fine_tuning.jobs.create(
    method={"type": "sft"},
    training_file=training_file_id,
    model="HuggingFaceTB/SmolLM2-135M-Instruct"  # Small model for quick training
)

print(f"🎯 Training job created: {job.id}")
print(f"Status: {job.status}")
job_id = job.id

## 📊 Monitor Training Progress with Real-time Logs

Arbor now provides built-in log monitoring! You can watch training progress in real-time.

In [ ]:
# Option 1: Watch job with real-time logs (recommended!)
print("🎬 Starting real-time job monitoring...")
arbor.watch_job(job_id, max_time=300, show_logs=True)  # Watch for 5 minutes max

In [ ]:
# Option 2: Traditional polling approach
import time

# Check job status manually
job_status = client.fine_tuning.jobs.retrieve(job_id)
print(f"📊 Job Status: {job_status.status}")

# Monitor until completion (with timeout)
timeout = 300  # 5 minutes
start_time = time.time()

while job_status.status in ['queued', 'running'] and (time.time() - start_time) < timeout:
    time.sleep(10)  # Check every 10 seconds
    job_status = client.fine_tuning.jobs.retrieve(job_id)
    elapsed = int(time.time() - start_time)
    print(f"⏱️  [{elapsed}s] Status: {job_status.status}")

print(f"\n🎉 Final Status: {job_status.status}")
if job_status.fine_tuned_model:
    print(f"✅ Fine-tuned model: {job_status.fine_tuned_model}")

### 🆚 Monitoring Options

**Option 1: `arbor.watch_job()`** - Shows real-time training logs as they happen ✨  
**Option 2: Traditional polling** - Just checks status periodically

The new `watch_job()` function gives you much better visibility into what's happening during training!

## 🧪 Test Fine-tuned Model

If training completed successfully, let's test the fine-tuned model.

In [ ]:
if job_status.status == 'succeeded' and job_status.fine_tuned_model:
    # Test the fine-tuned model
    response = client.chat.completions.create(
        model=job_status.fine_tuned_model,
        messages=[
            {"role": "user", "content": "What is reinforcement learning?"}
        ],
        temperature=0.7,
        max_tokens=150
    )
    
    print("🎭 Fine-tuned model response:")
    print(response.choices[0].message.content)

    # Shutdown the inference job
    arbor.shutdown_job(job_status.fine_tuned_model)
else:
    print(f"⚠️  Training not completed yet. Status: {job_status.status}")
    if job_status.status == 'failed':
        print("❌ Training failed. Check the logs for details.")

## 🚀 Advanced: DSPy GRPO with Multi-GPU Configuration

DSPy's GRPO optimizer requires a multi-GPU setup where inference and training run on separate GPUs for optimal performance. This requires at least 2 GPUs.

**Note**: Standard Google Colab provides only 1 GPU, so you'll need:
- A local machine with multiple GPUs, or
- A cloud instance with multiple GPUs

For single GPU environments, stick with the SFT training shown above!

In [ ]:
# DSPy GRPO with GPU Memory Sharing Example
import dspy
import random

# Sample classification data (banking domain)
sample_classification_data = [
    dspy.Example(text="I want to transfer money to my friend", label="transfer").with_inputs("text"),
    dspy.Example(text="What is my account balance?", label="balance").with_inputs("text"), 
    dspy.Example(text="I lost my credit card", label="card_issues").with_inputs("text"),
    dspy.Example(text="How do I reset my PIN?", label="pin_change").with_inputs("text"),
    dspy.Example(text="When will my salary be deposited?", label="direct_deposit").with_inputs("text"),
    dspy.Example(text="I need to pay my bills", label="bill_payment").with_inputs("text"),
    dspy.Example(text="Can I increase my credit limit?", label="credit_limit").with_inputs("text"),
    dspy.Example(text="Where is the nearest ATM?", label="atm_location").with_inputs("text"),
]

# Create train and validation sets
random.Random(42).shuffle(sample_classification_data)
trainset = sample_classification_data[:6]
valset = sample_classification_data[6:]

print(f"📊 Created training set: {len(trainset)} examples")
print(f"📊 Created validation set: {len(valset)} examples")

# Define the classification task
CLASSES = ["transfer", "balance", "card_issues", "pin_change", "direct_deposit", "bill_payment", "credit_limit", "atm_location"]
classify = dspy.ChainOfThought(f"text -> label: Literal{CLASSES}")

# Set up DSPy with Arbor provider
from dspy.clients.lm_local_arbor import ArborProvider

provider = ArborProvider()
student_lm = dspy.LM(
    model=f"openai/arbor:Qwen/Qwen2-0.5B-Instruct",  # Use the same small model
    provider=provider, 
    temperature=0.7, 
    api_base=server_info['base_url'], 
    api_key="arbor"
)

# Create student classifier
student_classify = classify.deepcopy()
student_classify.set_lm(student_lm)

# Define evaluation metric
metric = (lambda x, y, trace=None: x.label == y.label)

print(f"🎯 Classification task: {len(CLASSES)} classes")
print("🔧 DSPy classifier set up with Arbor backend")

In [ ]:
# Configure GRPO optimizer with multi-GPU setup
from dspy.teleprompt.grpo import GRPO

# Training configuration with multi-GPU setup
train_kwargs = {
    'per_device_train_batch_size': 2,
    'temperature': 0.7,
    'beta': 0.02,
    'learning_rate': 1e-5,
    'gradient_checkpointing': False,
    'bf16': True,
    'lr_scheduler_type': 'constant_with_warmup',
    'max_prompt_length': None,
    'max_completion_length': None,
    'lora': False,
    'report_to': 'none',
    'log_completions': False,
}

# Create GRPO compiler
compiler = GRPO(
    metric=metric,
    multitask=True,
    num_dspy_examples_per_grpo_step=2,
    num_rollouts_per_grpo_step=2,
    exclude_demos=True,
    num_train_steps=10,
    num_threads=4,
    use_train_as_val=False,
    num_steps_for_val=5,
    train_kwargs=train_kwargs,
)

print("🚀 GRPO compiler configured with multi-GPU setup")
print("   - Type: Multi-GPU configuration")
print("   - Inference GPUs: 1 (runs VLLM server)")
print("   - Training GPUs: 1 (runs training process)")
print("   - Total GPUs required: 2")

In [ ]:
# Run GRPO optimization (this will take several minutes)
print("🏃 Starting GRPO optimization...")
print("This will:")
print("  1. Launch VLLM inference server on GPU 0")
print("  2. Start training process on GPU 1") 
print("  3. Optimize the classifier using GRPO learning")
print("")
print("⚠️  REQUIREMENTS:")
print("   - At least 2 GPUs available")
print("   - Standard Colab (1 GPU) will NOT work")
print("   - Use Colab Pro+, local multi-GPU, or cloud instance")
print("")

# Check if we have enough GPUs
import torch
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"🔍 Detected {gpu_count} GPU(s)")
    
    if gpu_count < 2:
        print("❌ GRPO requires at least 2 GPUs")
        print("   Skipping GRPO demonstration")
    else:
        print("✅ Sufficient GPUs detected for GRPO")
        
        # Compile the classifier with GRPO
        classify_optimized = compiler.compile(
            student=student_classify,
            trainset=trainset,
            valset=valset,
        )
        
        print("✅ GRPO optimization completed!")
        print("📈 Your classifier has been optimized using reinforcement learning")
else:
    print("❌ No CUDA GPUs detected")
    print("   GRPO requires GPU acceleration")

## 🛠️ Server Management

Arbor provides utilities to check server status and manage the background process.

In [ ]:
# Check server status
status = arbor.status()
if status:
    print("🌳 Arbor server is running:")
    print(f"   Host: {status['host']}")
    print(f"   Port: {status['port']}")
    print(f"   Base URL: {status['base_url']}")
    print(f"   Storage: {status['storage_path']}")
    print(f"   GPU IDs: {status['gpu_ids']}")
else:
    print("❌ No Arbor server running")

## 🎯 Next Steps

Now that you've got Arbor running, here are some things to try:

1. **Experiment with different models**: Try larger models like `Qwen/Qwen2-0.5B-Instruct`
2. **Use your own data**: Upload your own JSONL training files
3. **Try GRPO training**: Use the GRPO example above if you have multiple GPUs
4. **Integrate with DSPy**: Use Arbor as a backend for DSPy programs
5. **Monitor training**: Use the monitoring endpoints to track GPU usage and training metrics

### 📚 Resources

- [Arbor GitHub Repository](https://github.com/Ziems/arbor)
- [DSPy RL Optimization Examples](https://dspy.ai/tutorials/rl_papillon/)
- [DSPy Discord Community](https://discord.gg/ZAEGgxjPUe)

### 💡 Pro Tips

**For Single GPU (Standard Colab):**
- Use `arbor.init()` at the start of your notebook - it's designed to be safe to call multiple times
- SFT training works great with single GPU
- The server will auto-cleanup when your session ends
- Use `arbor.shutdown_job()` to free up GPU resources when switching models

**For Multi-GPU (Colab Pro+, Local, Cloud):**
- GRPO training requires at least 2 GPUs (1 for inference, 1 for training)
- Use the advanced GRPO examples for reinforcement learning from preferences
- Monitor your GPU usage to optimize resource allocation
- Consider larger models and batch sizes with more GPU memory

### 🔧 GPU Requirements Summary

| Feature | GPU Requirement | Environment |
|---------|----------------|-------------|
| **Standard Training** | 1 GPU | ✅ Colab, Local, Cloud |
| **Model Inference** | 1 GPU | ✅ Colab, Local, Cloud |
| **GRPO Training** | 2+ GPUs | ❌ Colab, Local, Cloud |

Happy fine-tuning! 🌳✨